In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/cic-ids/cleaned_data1.csv')

# Feature Engineering 

In [ ]:
df.replace(to_replace=["Bot", "FTP-BruteForce", "SSH-Bruteforce","DoS attacks-GoldenEye","DoS attacks-Slowloris"], value="Malicious", inplace=True)

In [ ]:
df.replace(to_replace="Benign", value=0, inplace=True)
df.replace(to_replace="Malicious", value=1, inplace=True)

In [ ]:
columns_to_drop = [
    'Bwd URG Flags','Bwd Pkts/b Avg','Bwd Blk Rate Avg','Fwd Pkts/b Avg','CWE Flag Count','Fwd URG Flags','Bwd PSH Flags','Fwd Byts/b Avg','Bwd Byts/b Avg','Fwd Blk Rate Avg','Protocol_17','Pkt Len Min','FIN Flag Cnt','Active Std','SYN Flag Cnt'
]

In [ ]:
df.drop(columns = columns_to_drop,inplace=True)

# Scaling

In [ ]:
y = df['Label']
X = df.drop(['Label'],axis=1)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

scaler = StandardScaler()
x = scaler.fit_transform(X)
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=42)

# Decision Tree K fold with Boosting

In [ ]:
import pickle

In [ ]:
with open('/content/drive/MyDrive/TrainedModels/decisiontTree.pkl', 'rb') as f:
  clf = pickle.load(f)

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn import model_selection


kfold = model_selection.KFold(n_splits=10)

# Create an AdaBoost classifier with 100 decision tree models
boosting_clf = AdaBoostClassifier(base_estimator=clf, n_estimators=100, random_state=42)
results = model_selection.cross_val_score(boosting_clf, x_test, y_test, cv=kfold)
# Fit the AdaBoost classifier on the training data
print(results.mean())

0.9999618530162252


# SVM with K-fold with Boosting

In [ ]:
with open('/content/drive/MyDrive/TrainedModels/svm.pkl', 'rb') as f:
  svm = pickle.load(f)

In [ ]:


from numpy import mean
from numpy import std
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

cv = KFold(n_splits=10, random_state=1, shuffle=True)
scores = cross_val_score(svm, x_test, y_test, scoring='accuracy', cv=cv, n_jobs=-1)

print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

Accuracy: 0.941 (0.001)


# Voting Classifier for both Decision Tree and K-Fold 

In [ ]:
from sklearn.ensemble import VotingClassifier
kfold = model_selection.KFold(n_splits=10, random_state=1, shuffle=True)

ensemble = VotingClassifier(estimators=[('svm', svm), ('clf', clf)], voting='hard')
results = model_selection.cross_val_score(ensemble, x_test, y_test, cv=kfold)
print(results.mean())


/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warning

0.9529615587107576
